In [2]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
from util import random_word
from util import find_random_question
import re
import random

In [130]:
total_cost = 0
default_model = "gpt-4o-mini"
verbose = True
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.7, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

main_title = random_word()
if verbose:
    print(main_title)

problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 단어 1개를 제시하라."),
        HumanMessage(content="주제어: " + main_title + " 단어: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
word = word_responses[0][1]
def clean_text(text):
    text = text.replace("'", '＇')
    text = text.replace('"', '＇')
    
    pattern = r':\s*(.*)$'
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
    return text.strip()
word = clean_text(word)
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))

keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_12'
random_question = find_random_question(keyword, problem_type, detail_type=1, count=2, api_key_path="./../API_KEY/topik_api_key.json")
if verbose:
    print(random_question)


def split_text(input_text):
    # <보기>로 문자열을 분리하여 리스트로 만듭니다.
    parts = re.split(r'(<보기>)', input_text)
    
    # 중복된 <보기>를 제거하고 문장과 <보기>를 분리하여 저장
    result = []
    temp_text = ''
    
    for part in parts:
        if part == '<보기>':
            if temp_text.strip():
                result.append(temp_text.strip())
                temp_text = ''
            if '<보기>' not in result:
                result.append(part)
        else:
            temp_text += part
    
    # 마지막으로 남은 텍스트를 추가
    if temp_text.strip():
        result.append(temp_text.strip())
    
    example_text = result[0]
    answer_text = result.pop()
    return example_text, answer_text

def remove_parentheses_only(text):
    # 괄호만 제거하는 정규식
    text = re.sub(r'\s*\((.*?)\)\s*', r' \1 ', text)
    # 연속된 공백을 하나로 정리
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def fill_in_blank(text, answer, position):
    # 정답을 빈칸에 채워넣기
    text = text.replace(position, answer)
    # 나머지 빈칸을 정규식으로 제거하기
    text = re.sub(r'\( *㉠ *\)', '', text)
    text = re.sub(r'\( *㉡ *\)', '', text)
    text = re.sub(r'\( *㉢ *\)', '', text)
    text = re.sub(r'\( *㉣ *\)', '', text)
    
    return remove_parentheses_only(text)
example_dict = []
for question in random_question:
    example_text, answer_text =  split_text(question['example'])
    sentence = fill_in_blank(example_text, answer_text, question['answer'])
    if verbose:
        print(example_text)
        print(answer_text)
        print("origin_text")
        print(sentence)
    problem_generate_model.request_models_responses([
        SystemMessage(content="문장의 주제 단어 1개를 제시하라."),
        HumanMessage(content="문장: "+ sentence + "주제 단어: "),
            ]
        )
    total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_generate_model.get_model_responses()[0][2]).group(1))
    title_responses = problem_generate_model.get_model_responses()[0][1]
    title = clean_text(title_responses)
    temp = {"주제어":title, "문장": sentence, "선택지":question['selector']}
    example_dict.append(temp)
        
    if verbose:
        print("example_dict")
        print(example_dict)

    example_str = ""
    for example in example_dict:
        example_str += (
            "주제어: " + example["주제어"] + " 결과: " + example["문장"] + " "
        )
    if verbose:
        print("example_str")
        print(example_str)
        
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content="#예시처럼 주어지는 주제에 대한 글을 제시하라. 문장은 6개로 이루어지며 이어지는 순서가 명확해야 한다. #답변 예시 "
            + example_str
        ),
        HumanMessage(content="주제: " + word + " 문장: "),
    ]
)
example_responses = problem_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', example_responses[0][2]).group(1))

example_responses = example_responses[0][1]
text = example_responses
text = clean_text(text)
if verbose:
    print(text)
origin_text = text
text = [s + "." for s in text.split(".") if s]

selectors = ["㉠", "㉡", "㉢", "㉣"]
selector_answer = random.randint(0, 3)
answer = text[1:][selector_answer]

example = ""
i=0
text.remove(answer)
for item in text:
    if i > 3:
        example += item
    else:
        example += f"{item} ({selectors[i]})"
    i += 1

result = {
            "example": str([[answer.strip()],[example.strip()]]),
            "selector": selectors,
            "answer": selectors[selector_answer],
            "eval_answer": selectors[selector_answer],
            "eval_explain":origin_text,
            "total_cost": total_cost,
        }
display(result)

수요일
gpt-4o-mini | 
주제어: 수요일  
단어: 주간

OpenAI Tokens Used: 53
    Prompt Tokens: 41
    Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $4e-05
Total Cost (KRW): ₩0.054
--------------------------------------------------------------------------------------------------
[{'example': '동궐도는 창덕궁과 창경궁 전체를 그린 조선 시대의 그림이다. ( ㉠ ) 세로 2m, 가로 5m가 넘는 대작으로 건축물은 물론 주변의 산과 궁궐 안 연못, 나무까지 그대로 그려 넣었다. 건물 배치와 건물 사이의 거리도 완벽하게 재현했다. ( ㉢ ) 이러한 이유로 제작자와 제작 연도를 포함하여 그림에 관련된 정보 일체가 왕실 기밀이었을 것으로 추정된다. ( ㉣ ) <보기> 이처럼 정밀하면서도 실제와 같은 그림은 외부인이 궁궐에 침입할 목적으로 사용할 수 있다.', 'selector': ['㉠', '㉡', '㉢', '㉣'], 'answer': '㉢'}, {'example': '이영천의 [다시, 오래된 다리를 거닐다]는 다리를 소재로 한국인의 삶을 돌아본 책이다. ( ㉠ ) 여기에서는 수백 년 전에 놓인 징검다리, 왕을 위한 다리 등을 통해 우리 고유의 풍속과 역사를 만난다. ( ㉡ ) 이어서 근현대식 다리와 함께 기술의 발전이 다리와 우리 사회에 가져온 변화를 살펴본다. ( ㉢ ) 이 책은 과거와 현재의 다리를 거닐며 우리가 지나온 길과 가야 할 길을 생각해 보게 한다. ( ㉣ ) <보기> 이 책은 크게 두 부분으로 구성되어 있는데 먼저 옛 다리들과 그에 얽힌 이야기를 다룬다.', 'selector': ['㉠', '㉡', '㉢', '㉣'], 'answer': '㉠'}]
동궐도는 창덕궁과 창경궁 전체를 그린 조선 시대의 그림이다. ( ㉠ ) 세로 2

{'example': "[['각 주는 일상적인 업무 및 활동을 계획하고 조정하는 데 중요한 역할을 한다.'], ['주간은 일주일을 의미하며, 매주 반복되는 주기적인 시간 단위이다. (㉠) 일반적으로 주는 월요일부터 시작하여 일요일로 끝나는 형태로 구성된다. (㉡) 특히, 주간 계획표를 통해 개인의 목표와 일정을 관리할 수 있다. (㉢) 또한, 많은 사람들은 주말을 이용해 휴식과 여가 활동을 즐기며 에너지를 재충전한다. (㉣) 이처럼 주간은 우리 삶의 리듬을 형성하고, 효율적인 시간 관리를 가능하게 한다.']]",
 'selector': ['㉠', '㉡', '㉢', '㉣'],
 'answer': '㉡',
 'eval_answer': '㉡',
 'eval_explain': '주간은 일주일을 의미하며, 매주 반복되는 주기적인 시간 단위이다. 일반적으로 주는 월요일부터 시작하여 일요일로 끝나는 형태로 구성된다. 각 주는 일상적인 업무 및 활동을 계획하고 조정하는 데 중요한 역할을 한다. 특히, 주간 계획표를 통해 개인의 목표와 일정을 관리할 수 있다. 또한, 많은 사람들은 주말을 이용해 휴식과 여가 활동을 즐기며 에너지를 재충전한다. 이처럼 주간은 우리 삶의 리듬을 형성하고, 효율적인 시간 관리를 가능하게 한다.',
 'total_cost': 0.9450000000000001}

In [10]:
text = "배우는 영화, 드라마, 연극 등 다양한 매체에서 역할을 맡아 연기하는 예술가이다. 그들은 각기 다른 캐릭터를 통해 관객에게 감정을 전달하고 이야기를 풀어내는 중요한 역할을 한다. 배우는 종종 대본을 분석하고 캐릭터에 대한 깊은 이해를 바탕으로 연기를 준비한다. 또한, 연기력뿐만 아니라 목소리, 몸짓, 표정 등 다양한 표현력을 필요로 한다. 많은 배우들이 오디션을 통해 기회를 얻고, 이를 통해 자신의 경력을 쌓아간다. 결국, 배우는 관객과의 교감을 통해 그들의 삶에 감동을 주는 직업이다."
text_list = text.split(".")[:-1]
print(text_list)
print(random.choice(text_list[1:]))

['배우는 영화, 드라마, 연극 등 다양한 매체에서 역할을 맡아 연기하는 예술가이다', ' 그들은 각기 다른 캐릭터를 통해 관객에게 감정을 전달하고 이야기를 풀어내는 중요한 역할을 한다', ' 배우는 종종 대본을 분석하고 캐릭터에 대한 깊은 이해를 바탕으로 연기를 준비한다', ' 또한, 연기력뿐만 아니라 목소리, 몸짓, 표정 등 다양한 표현력을 필요로 한다', ' 많은 배우들이 오디션을 통해 기회를 얻고, 이를 통해 자신의 경력을 쌓아간다', ' 결국, 배우는 관객과의 교감을 통해 그들의 삶에 감동을 주는 직업이다']
 결국, 배우는 관객과의 교감을 통해 그들의 삶에 감동을 주는 직업이다


In [111]:

text = [s + "." for s in text.split(".") if s]

selectors = ["㉠", "㉡", "㉢", "㉣"]
selector_answer = random.randint(0, 3)
answer = text[1:][selector_answer]

example = ""
i=0
text.remove(answer)
for item in text:
    if i > 3:
        example += item
    else:
        example += f"{item} ({selectors[i]})"
    i += 1
    
    
print(example)
print(answer)
print(selectors[selector_answer])

배우는 영화, 드라마, 연극 등 다양한 매체에서 역할을 맡아 연기하는 예술가이다. (㉠) 
그들은 각기 다른 캐릭터를 통해 관객에게 감정을 전달하고 이야기를 풀어내는 중요한 역할을 한다. (㉡) 
배우는 종종 대본을 분석하고 캐릭터에 대한 깊은 이해를 바탕으로 연기를 준비한다. (㉢) 
많은 배우들이 오디션을 통해 기회를 얻고, 이를 통해 자신의 경력을 쌓아간다. (㉣) 
결국, 배우는 관객과의 교감을 통해 그들의 삶에 감동을 주는 직업이다.
 
또한, 연기력뿐만 아니라 목소리, 몸짓, 표정 등 다양한 표현력을 필요로 한다.
㉢
